In [2]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

# My added modules:
############
import pandas as pd
import numpy as np
from datetime import datetime
############

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

In [20]:
HANGMAN_URL = "filler"

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
        # My added attributes:
        #####################################################
        
        # Variables to prevent unnecessary and time consuming / computationally expensive repetition within
        # guess function
        self.first_time = True
        self.prev_word = ""
        self.total_count = collections.Counter()
        
        
        """ 
            Reads in dataframe used to keep track of common regular expressions so that letter counts can just
            be looked up instead of recalculated by the guess function.
            This was an attempt to reduce the number of times the guess function would have to loop through the entire
            dictionary and to speed the program up. Unfortunately, as I suspected, not enough expressions came up
            regularly enough in the dictionary to really make this worth it. 
        
        """
        # MAY NEED TO CHANGE THIS FILE LOCATION
        self.regex_df_loc = "C:/Users/alexh/OneDrive/Documents/Coding/Python/Careers/filler/regexes.pkl"
        self.regex_df = pd.read_pickle(self.regex_df_loc)
        clean_regex_df_index = self.regex_df.index.drop_duplicates()
        self.regex_df = self.regex_df.loc[clean_regex_df_index]
        self.whole_game_regexes = []

        #################################################
        
        
    def guess(self, word, weights):        
        
        """
            Method that takes the hangman word and returns a letter to guess
        """
        
        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")
        len_word = len(clean_word)
        
        previous_word = self.prev_word
        self.prev_word = clean_word
        
        # Variables to check if we want to use the baseline example method provided
        num_guessed_letters = len(clean_word.replace(".", ""))
        proportion_guessed_letters = num_guessed_letters / len_word
        len_current_dictionary = len(self.current_dictionary)
        
        # Use the provided example method under the following conditions
        if (len_current_dictionary > weights[0] and proportion_guessed_letters < weights[1])\
            or len_word <= 3:
                
            guess_letter = self.dictionary_average(len_word, clean_word)
            return guess_letter                
        
        else:
            
            """
            This method produces all possible regular expressions found from the known letters of the
            unknown word (while taking into account whether or not the known letters are at the start
            or end of the word), tabulates all of the letters that would fill the positions of the
            unknown letters in each regular expression and combines the total from each regular
            expression into a weighted sum in order to get the optimal letter to guess. The weighting
            is done based on the number of known letters in the regular expression.
            """
            
            # checks to see if the hangman word has changed since we last saw it, if not, we don't
            # have to recalculate all of the regular expression matches in the dictionary
            if not previous_word == clean_word or self.first_time:
                self.first_time = False
                
                # Produce a series of regular expressions based on the hangman word
                regexes = self.produce_regexes(len_word, clean_word)
                
                # Resetting a variable to check if any new regular expressions
                # were actually encountered and need to be written to the DataFrame
                entered = False                
                
                # Variables to add new calculated regular expressions to the pandas dataframe
                total_count = collections.Counter()
                pandas_list = []
                
                for expression in regexes:
                    
                    # add regular expression to list of regular expressions for the whole game
                    self.whole_game_regexes.append(expression)
                    
                    # Check reference dataframe to see if counter has been pre-calculated
                    if expression in self.regex_df.index:
                        counter = self.regex_df.loc[expression, "letter_counter"]
                    
                    # If not, calculate the counter yourself
                    else:
                        
                        entered = True
                        
                        # Find total number of letters appearing in all dictionary words
                        # fitting a regular expression
                        counter = self.get_counter(expression)
                        
                        # Update list with regular expressions to later add to the DataFrame
                        pandas_list.append({"Regex": expression,
                                            "letter_counter": counter, "counter": 0})
                    
                    
                    # Figure out the weighting to apply to the collections.Counter
                    # object for the expression based on the number of known letters in the
                    # regular expression 
                    
                    # Find weighting
                    num_letters = sum(c.isalpha() for c in expression)
                    num_letters = 6 if num_letters > 6 else num_letters
                    weighting = weights[num_letters + 1]
                    
                    # Take into account weighting 
                    temp_counter = collections.Counter()
                    for key, value in counter.items():
                        temp_counter[key] = value * weighting
                    total_count.update(temp_counter)
                
                    
                    self.total_count = total_count.most_common()
                
                # Check if new regular expressions were encountered if so, add these new regular
                # expressions to the reference data frame
                if entered:
                    df = pd.DataFrame(pandas_list)
                    df = df.set_index("Regex")
                    self.regex_df =\
                        pd.concat([self.regex_df, df])
                    
            
            
            # Choose the guess letter with the highest count, if all letters have been guessed,
            # use the total training dictionary distribution once more
            
            guess_letter = '!'
            
            for letter,instance_count in self.total_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break
            
        
            if guess_letter == '!':
                sorted_letter_count = self.full_dictionary_common_letter_sorted
                for letter,instance_count in sorted_letter_count:
                    if letter not in self.guessed_letters:
                        guess_letter = letter
                        break     
            
            return guess_letter
        
    
    # Baseline method provided
    def dictionary_average(self, len_word, clean_word):
        
        # Initialize new possible words dictionary to empty
        new_dictionary = []
        
        # iterate through all of the words in the previous plausible dictionary
        for dict_word in self.current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word, dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   

        guess_letter = '!'
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter
    
    
    # Calculates all possible regular expressions formed surrounding unknown letters.
    # (E.g. for word 'l e _ _ _ a ' all regular expressions would be:
    # '^le..', '^le.', '.a$', '...a$', '..a$', '^le...a$', 'e...a$', '^le...'
    # catching unwanted letters in words that match the regular expression, 
    def produce_regexes(self, len_word, clean_word):
        
        regexes = []
        for i in range(len_word):
            for j in range(i, len_word + 1):
                regex = clean_word[i:j]
                if "." in regex:
                    if i == 0:
                        if j == len_word:
                            regexes.append("^" + clean_word[i:j] + "$")
                        else:
                            regexes.append("^" + clean_word[i:j])
                    if i > 0:
                        if j==len_word:
                            regexes.append(clean_word[i:j] + "$")                        
                        else:
                            regexes.append(clean_word[i:j])
                            
        # Remove regular expressions that only have one letter or less (they're not useful)
        regexes = [item for item in regexes if len(item.replace(".", "")) > 1]
        # Removing potential duplicates 
        regexes = list(set(regexes))        
        return regexes
    
    
    # Returns collections.Counter object for total number of letters appearing in all
    # dictionary words fitting a regular expression. Only guesses letters that occupy 
    # the position the "." would take. 
    def get_counter(self, expression):
        
        # Find positions of unknown letters in the expression
        cleaned_expression = expression.replace("^", "").replace("$", "")
        indexes = [i for i, ltr in enumerate(cleaned_expression) if ltr == "."]
        
        # Checking if the word matches the regular expression and taking
        # the letter values in and immediately around where the regular
        # expression matches
        all_matching_letters = []
        for dict_word in self.full_dictionary:
            re_obj = re.search(expression, dict_word)
            if re_obj:
                a, b = re_obj.span()
                matching_letters = [dict_word[a + idx] for idx in indexes]
                all_matching_letters.append("".join(matching_letters))
    
        LETTERS = "".join(all_matching_letters)    
        return collections.Counter(LETTERS)
        
        

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        
        # My additions:
        #######################
        # Reset variables corresponding to line 132
        self.prev_word = ""
        self.first_time = True
        
        # Resets list to keep track of all of the regular expressions that come up throughout the game
        self.whole_game_regexes = []
        
        # This set of weights worked well in the end, though I have no idea if they are close to optimal
        weights = [50, 0.3, 1, 2, 4, 8, 12, 20]
        #######################
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word, weights)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Server response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    
                    # ADDED CODE
                    ##################################

                    # Update the pandas DataFrame at the end of the game with the new regular expression counts:

                    # Removing duplicates
                    duplicates_removed = set(self.whole_game_regexes)
                    for regex in duplicates_removed:
                        if self.regex_df.loc[regex, "counter"] == 0:
                            self.regex_df.loc[regex, "counter"] = 1
                        else:
                            self.regex_df.loc[regex, "counter"] = self.regex_df.loc[regex, "counter"] + 1



                    # Write DataFrame of all encountered regexes to external file for safekeeping
                    self.regex_df.to_pickle(self.regex_df_loc)
                        
                    ##################################    
                    
                    return True
                
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    
                    # ADDED CODE
                    #################################

                    # Update the pandas DataFrame at the end of the game with the new regular expression counts:

                    # Removing duplicates
                    duplicates_removed = set(self.whole_game_regexes)
                    for regex in duplicates_removed:
                        if self.regex_df.loc[regex, "counter"] == 0:
                            self.regex_df.loc[regex, "counter"] = 1
                        else:
                            self.regex_df.loc[regex, "counter"] = self.regex_df.loc[regex, "counter"] + 1



                    # Write DataFrame of all encountered regexes to external file for safekeeping
                    self.regex_df.to_pickle(self.regex_df_loc)
                        
                    #################################
                    
                    return False
                
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        num_retry, time_sleep = 5, 2                                                                                        
        for it in range(num_retry):                                                                                         
            try:                                                                                                            
                response = self.session.request(                                                                            
                    method or "GET",                                                                                        
                    HANGMAN_URL + path,                                                                                     
                    timeout=self.timeout,                                                                                   
                    params=args,                                                                                            
                    data=post_args                                                                                          
                )                                                                                                           
                break                                                                                                       
            except requests.HTTPError as e:                                                                                 
                response = json.loads(e.read())                                                                             
                raise HangmanAPIError(response)                                                                             
            except requests.exceptions.SSLError as e:                                                                       
                if it + 1 == num_retry:                                                                                     
                    raise                                                                                                   
                time.sleep(time_sleep)  

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
    # =============================================================================
    #     Below are newly defined methods created in order to try and find the optimal weights for the above
    #     guess method. I included these here just as a demonstration of my thought processes and problem solving
    #     procedure. For the actual guessing method above they serve no purpose, and they won't do anything without
    #     the self-built start_game function I made, written elsewhere (I can send it if you would like, but it's
    #     not that interesting).
    # =============================================================================
        
    
    def cost_function(self, weights):
        """
        A cost function based on the accuracy of my guess method for a particular set of 'weights' -
        which are key values that come up in that method that could be varied to produce different results.
        The cost_function is literally (1 - accuracy) with accuracy measured as fraction of correctly guessed
        words within the permitted tries_remains. Unfortunately even after 100 repeats for the same weights I
        found the cost calculated varied over a range of 0.08, which wasn't good enough for my gradient descent
        method detailed below.
        """
        
        success_counter = 0
        for i in range(100):
            
            if self.start_game(weights, verbose=False):
                success_counter += 1
                print(f"Game {i}: SUCCESS")
            else:
                print(f"Game {i}: FAILURE")
        return 1 - success_counter / 100
    
    
    def gradient_descent(self, learning_rate):
        """ 
            This very rudimentary gradient descent function is something I designed because I didn't really quite
            know what machine learning tools from what package to use to apply to this problem. Despite my best
            efforts the cost function would just take too long, and if the repeats were made shorter then there would
            be too much error in the accuracy.
        """
        
        # Initialising starting weights and costs, 
        #weights = np.random.randint(0, 1000, size=8)
        #weights = input("Enter your eight selected weights: ")
        weights = [50, 0.3, 1, 2, 4, 8, 12, 20]
        
        # Initial cost function return value
        old_cost = 1
        
        # Storage for the different weights and learning costs 
        cost_to_weights = dict()
        
        # Loops while the gradient descent hasn't failed to improve it's accuracy five times in a row
        consecutive_failures = 0
        while consecutive_failures < 5:
            
            # A list to receive the new weights
            updated_weights = []
            
            # Iterating over each weight
            for i, weight in enumerate(weights):
                
                test_weights = weights.copy()
                test_weights[i] = weight * (1 - learning_rate)
                cost1 = self.cost_function(test_weights)
                print(f"Weight {i} changed from {weight} to {test_weights[i]}")
                print(f"Yields cost of {cost1}")
                
                test_weights = weights.copy()
                test_weights[i] = weight * (1 + learning_rate)
                cost2 = self.cost_function(test_weights)
                print(f"Weight {i} changed from {weight} to {test_weights[i]}")
                print(f"Yields cost of {cost2}")
                
                # Pick whichever direction improves the cost most to adjust the weighting and append
                # it to the updated weights list
                if cost1 > cost2:
                    updated_weights.append(weight * (1 + learning_rate))
                else:
                    updated_weights.append(weight * (1 - learning_rate))
            
            # Updating weights to those optimised in the above iteration of gradient descent
            weights = updated_weights
            updated_cost = self.cost_function(updated_weights)
            
            # Cheecking to see if there has been significant improvement in the cost function
            if updated_cost + 0.001 < old_cost:
                print("Cost function improved")
                print("Updated weights: {updated_weights}")
                old_cost = updated_cost
                consecutive_failures = 0
            else:
                consecutive_failures += 1
                print("Cost function failed to improve. Consecutive failures: {consecutive_failures}")
            
            # Slowing the learning rate for better convergence
            learning_rate *= 0.99
            
        return weights
            
    def trees(self):
        """
            Generates random weights, then finds the accuracy of the model for these random weights. Plan was then to
            use ML algorithm to figure out relationship between the weights as input and the accuracy as a label.
            This is clearly an inferior method compared to my gradient descent method, but it could in theory take
            fewer repeats to still get at an answer that would improve the weighting selection.
            Unfortunately, there were similar problems with time and accuracy of the cost function to be realistic to
            implement on my laptop.
        
        """
        cost_to_weights = dict()
        i = 0
        while i < 100:
            weights = []
            weights.append(np.random.lognormal(6, 3))
            weights.append(np.random.randint(0, 7))
            for i in range(2, 8):
                weights.append(np.random.randint(0, 100))
            cost = self.cost_function(weights)
            cost_to_weights[cost] = weights
            print(weights, ": ", cost)
        return cost_to_weights
    
    
    
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [21]:
api = HangmanAPI(access_token="fb3c0a9c504726869131355169dae9", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [27]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
print('run %d practice games out of an allotted 100,000' %total_practice_runs)


Successfully start a new game! Game ID: e56c9fb56cb0. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': 'e56c9fb56cb0', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e _ _ _ _ _ _ _ _ e _ '}
Guessing letter: d
Server response: {'game_id': 'e56c9fb56cb0', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ _ _ _ _ _ _ e _ '}
Guessing letter: r
Server response: {'game_id': 'e56c9fb56cb0', 'status': 'ongoing', 'tries_remains': 5, 'word': 'r e _ _ r _ _ _ _ _ e _ '}
Guessing letter: t
Server response: {'game_id': 'e56c9fb56cb0', 'status': 'ongoing', 'tries_remains': 5, 'word': 'r e _ _ r _ t _ _ _ e _ '}
Guessing letter: a
Server response: {'game_id': 'e56c9fb56cb0', 'status': 'ongoing', 'tries_remains': 4, 'word': 'r e _ _ r _ t _ _ _ e _ '}
Guessing letter: i
Server response: {'game_id': 'e56c9fb56cb0', 'status': 'ongoing', 'tries_remains': 4, 'word': 'r e _ _ r _ t i _ i e _ '}
Guessing letter: o
Server response: {'game_

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [71]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

Playing  378  th game
Playing  379  th game
Playing  380  th game
Playing  381  th game
Playing  382  th game
Playing  383  th game
Playing  384  th game
Playing  385  th game
Playing  386  th game
Playing  387  th game
Playing  388  th game
Playing  389  th game
Playing  390  th game
Playing  391  th game
Playing  392  th game
Playing  393  th game
Playing  394  th game
Playing  395  th game
Playing  396  th game
Playing  397  th game
Playing  398  th game
Playing  399  th game
Playing  400  th game
Playing  401  th game
Playing  402  th game
Playing  403  th game
Playing  404  th game
Playing  405  th game
Playing  406  th game
Playing  407  th game
Playing  408  th game
Playing  409  th game
Playing  410  th game
Playing  411  th game
Playing  412  th game
Playing  413  th game
Playing  414  th game
Playing  415  th game
Playing  416  th game
Playing  417  th game
Playing  418  th game
Playing  419  th game
Playing  420  th game
Playing  421  th game
Playing  422  th game
Playing  4

Playing  751  th game
Playing  752  th game
Playing  753  th game
Playing  754  th game
Playing  755  th game
Playing  756  th game
Playing  757  th game
Playing  758  th game
Playing  759  th game
Playing  760  th game
Playing  761  th game
Playing  762  th game
Playing  763  th game
Playing  764  th game
Playing  765  th game
Playing  766  th game


HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [28]:
[total_practice_runs,total_recorded_runs,total_recorded_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.479
